In [3]:
import pandas as pd

# Assuming ids_df and other_df are your DataFrames
ids_path = '/Users/ryderwishart/text-librarian/data/marble/ids.txt'
ids_df = pd.read_csv(ids_path, sep='\t', header=0, dtype=str)

# other_df is a concat of two TSVs
macula_paths = [
    '/Users/ryderwishart/librarian-api/data/macula/macula-hebrew.tsv',
    '/Users/ryderwishart/librarian-api/data/macula/macula-greek-SBLGNT.tsv'
]

macula_dfs = [pd.read_csv(path, sep='\t', header=0, dtype=str) for path in macula_paths]

other_df = pd.concat(macula_dfs)

print(ids_df.head())
print(other_df.head()) 




         MARBLE_ID       TEXT     VREF
0  H00100100100002        בְּ  GEN 1:1
1  H00100100100004  רֵאשִׁ֖ית  GEN 1:1
2  H00100100100006    בָּרָ֣א  GEN 1:1
3  H00100100100008  אֱלֹהִ֑ים  GEN 1:1
4  H00100100100010       אֵ֥ת  GEN 1:1
          xml:id        ref class       text transliteration after  \
0  o010010010011  GEN 1:1!1  prep        בְּ              bə   NaN   
1  o010010010012  GEN 1:1!1  noun  רֵאשִׁ֖ית         rēʾšiyṯ         
2  o010010010021  GEN 1:1!2  verb    בָּרָ֣א           bārāʾ         
3  o010010010031  GEN 1:1!3  noun  אֱלֹהִ֑ים        ʾĕlōhiym         
4  o010010010041  GEN 1:1!4    om       אֵ֥ת             ʾēṯ         

  strongnumberx stronglemma sensenumber     greek  ... normalized strong case  \
0         0871a         בְּ         NaN        ἐν  ...        NaN    NaN  NaN   
1          7225    רֵאשִׁית           1      ἀρξῇ  ...        NaN    NaN  NaN   
2          1254      בָּרָא           1  ἐποίησεν  ...        NaN    NaN  NaN   
3          0430    אֱ

In [ ]:
# Sort both DataFrames by the 'TEXT' column
ids_df_sorted = ids_df.sort_values(by='TEXT')
other_df_sorted = other_df.sort_values(by='text')

# Merge the two DataFrames based on the sorted 'TEXT' and 'text' columns
merged_df = pd.merge(ids_df_sorted, other_df_sorted, left_on='TEXT', right_on='text', how='left')

# Rename the 'xml:id' column to 'MACULA_ID'
merged_df.rename(columns={'xml:id': 'MACULA_ID'}, inplace=True)

# Drop the 'text' column from the merged DataFrame
merged_df.drop(columns=['text'], inplace=True)

# Print the updated merged DataFrame
print(merged_df)